# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_INDEX, MODELNET10_64_INDEX, MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

VAE Training Notes:

* Higher KL weight results in network focusing less on minimizing reconstruction loss and thus produces lower quality reconstructions
* KL weight < .1 sees diminishing gains; use kl == .1
* Reconstruction loss is very important
* Latent dim of 100 borrowed from reference paper
* Latent dim of 1000 results in lower losses but overtrains (also, reconstructions do not look as good)
* TODO: experiment with latent dim of 350
* Learning rate of 0.0001 borrowed from reference papers; 0.001 trains faster but hits nan loss after a while

VAEGAN Training Notes:

* Architecture as-is (after adjusting loss functions from GAN loss to VAEGAN loss)

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
#INDEX = MODELNET10_TOILET_INDEX
#INDEX = MODELNET10_INDEX
INDEX = MODELNET10_64_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True,
                # desk is not availanle in 64dims
                'categories': ['bathtub', 'bed', 'chair', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet'],
                # bathtub  bed  chair  desk  dresser  monitor  night_stand  sofa  table  toilet
                # all categories
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 64,
                "batch_size": 128,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,  # 0.001 hits nan loss eventually
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 100,
                "keep_prob": 0.8,
                "kl_div_loss_weight": .1,
                "recon_loss_weight": 10000,
                "ll_weight": 1,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": True,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 5,
                "save_step": 5,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                #'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                #'tb_compare': [('vaegan_logless_loss', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54/tb')],
                #'tb_compare': [('voxel_vae_toilet_200epochs_1', os.path.join(MODEL_DIR, 'voxel_vae_toilet_200epochs_1', 'tb'))],
                #'tb_compare': [('vae_200epochs_p1kl', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-25_11-05-00/tb')],
                #'tb_compare': [('vae_toilet_nightstand_monitor', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-27_05-27-14/tb')],
                'no_gan': True,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1,
                'dis_noise': 0,
                'adaptive_lr': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-04-08_09-13-21
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.categories"
WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.adaptive_lr"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.dis_noise"
WARNING - root - Added new config entry: "cfg.model.display_step"

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-04-08_09-13__root.log
Starting train_vaegan main
Numpy random seed: 920882079
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-04-08_09-13-21/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_64_index.csv and pctile None
Shuffling dataset
Filtering dataset by categories: ['bathtub', 'bed', 'chair', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
N before filter: 71285
N after filter: 71285
dataset n_input=71285
Splitting Datasets
Num input = 71285
Num batches per epoch = 556.91
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 62, 62, 62, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 31, 31, 31, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 29, 29, 29, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 15, 15, 15, 64)
encoder/batch_normalization_3/batchnorm/ad

<tf.Variable 'encoder/conv3d_3/kernel/Adam:0' shape=(3, 3, 3, 32, 64) dtype=float32_ref>
<tf.Variable 'encoder/conv3d_3/kernel/Adam_1:0' shape=(3, 3, 3, 32, 64) dtype=float32_ref>
<tf.Variable 'encoder/conv3d_3/bias/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/conv3d_3/bias/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/gamma/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/gamma/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/beta/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/beta/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_mean/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_mean/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encod

<tf.Variable 'decoder/dec_conv1_64/kernel/Adam_1:0' shape=(3, 3, 3, 64, 1) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/gamma/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/gamma/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/beta/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/beta/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/moving_mean/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/moving_mean/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/moving_variance/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_1/moving_variance/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv2_64/kernel/Adam:0' shape=(3, 3, 3, 32, 64) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv2_64/kernel/Adam_1:0' shape=(3, 3

Memory Use (GB): 1.5547561645507812
Epoch: 0 / 100, Batch: 10 (0 / 1408), Elapsed time: 6.89 mins
Enc Loss = 780.12, KL Divergence = 0.00, Reconstruction Loss = 0.08, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 7.57 mins
Memory Use (GB): 1.5605278015136719
Epoch: 0 / 100, Batch: 11 (0 / 1536), Elapsed time: 7.57 mins
Enc Loss = 821.26, KL Divergence = 0.00, Reconstruction Loss = 0.08, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 8.25 mins
Memory Use (GB): 1.5593070983886719
Epoch: 0 / 100, Batch: 12 (0 / 1664), Elapsed time: 8.26 mins
Enc Loss = 634.67, KL Divergence = 0.01, Reconstruction Loss = 0.06, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 8.94 mins
Memory Use (GB): 1.5853614807128906
Epoch: 0 / 100, Batch: 13 (0 / 1792), Elapsed time: 8.95 mins
Enc Loss = 761.25, KL Divergence = 0.01, Reconstruction Loss = 0.08, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 9.62 mins


...

Epoch: 0 / 100, Batch: 175 (0 / 22528), Elapsed time: 119.68 mins
Enc Loss = 219.78, KL Divergence = 120.14, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 120.36 mins
Memory Use (GB): 1.8076400756835938
Epoch: 0 / 100, Batch: 176 (0 / 22656), Elapsed time: 120.36 mins
Enc Loss = 218.47, KL Divergence = 112.05, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 121.04 mins
Memory Use (GB): 1.8076133728027344
Epoch: 0 / 100, Batch: 177 (0 / 22784), Elapsed time: 121.04 mins
Enc Loss = 236.48, KL Divergence = 121.91, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 121.72 mins
Memory Use (GB): 1.8077888488769531
Epoch: 0 / 100, Batch: 178 (0 / 22912), Elapsed time: 121.73 mins
Enc Loss = 226.25, KL Divergence = 118.69, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 122.40 mins
Memo

Enc Loss = 212.52, KL Divergence = 132.23, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 143.61 mins
Memory Use (GB): 1.8190727233886719
Epoch: 0 / 100, Batch: 210 (0 / 27008), Elapsed time: 143.62 mins
Enc Loss = 206.42, KL Divergence = 129.67, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 144.30 mins
Memory Use (GB): 1.8190422058105469
Epoch: 0 / 100, Batch: 211 (0 / 27136), Elapsed time: 144.30 mins
Enc Loss = 218.42, KL Divergence = 129.87, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 144.98 mins
Memory Use (GB): 1.824188232421875
Epoch: 0 / 100, Batch: 212 (0 / 27264), Elapsed time: 144.98 mins
Enc Loss = 212.89, KL Divergence = 148.88, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 145.66 mins
Memory Use (GB): 1.8250312805175781
Epoch: 0 / 100, Batch: 213 (0 / 273

Enc Loss = 205.63, KL Divergence = 138.38, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 165.51 mins
Memory Use (GB): 1.8432121276855469
Epoch: 0 / 100, Batch: 242 (0 / 31104), Elapsed time: 165.52 mins
Enc Loss = 187.36, KL Divergence = 135.72, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 166.19 mins
Memory Use (GB): 1.8427810668945312
Epoch: 0 / 100, Batch: 243 (0 / 31232), Elapsed time: 166.20 mins
Enc Loss = 189.12, KL Divergence = 126.84, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 166.87 mins
Memory Use (GB): 1.8193473815917969
Epoch: 0 / 100, Batch: 244 (0 / 31360), Elapsed time: 166.88 mins
Enc Loss = 190.25, KL Divergence = 140.14, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 167.56 mins
Memory Use (GB): 1.8188858032226562
Epoch: 0 / 100, Batch: 245 (0 / 31

Enc Loss = 194.76, KL Divergence = 150.87, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 187.39 mins
Memory Use (GB): 1.821533203125
Epoch: 0 / 100, Batch: 274 (0 / 35200), Elapsed time: 187.40 mins
Enc Loss = 190.95, KL Divergence = 124.77, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 188.07 mins
Memory Use (GB): 1.821441650390625
Epoch: 0 / 100, Batch: 275 (0 / 35328), Elapsed time: 188.08 mins
Enc Loss = 190.89, KL Divergence = 140.26, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 188.76 mins
Memory Use (GB): 1.821319580078125
Epoch: 0 / 100, Batch: 276 (0 / 35456), Elapsed time: 188.76 mins
Enc Loss = 177.07, KL Divergence = 118.13, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 189.44 mins
Memory Use (GB): 1.8217048645019531
Epoch: 0 / 100, Batch: 277 (0 / 35584), 

Enc Loss = 178.83, KL Divergence = 132.64, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 209.31 mins
Memory Use (GB): 1.7626991271972656
Epoch: 0 / 100, Batch: 306 (0 / 39296), Elapsed time: 209.32 mins
Enc Loss = 187.91, KL Divergence = 152.95, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 210.00 mins
Memory Use (GB): 1.7626914978027344
Epoch: 0 / 100, Batch: 307 (0 / 39424), Elapsed time: 210.00 mins
Enc Loss = 188.06, KL Divergence = 136.67, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 210.68 mins
Memory Use (GB): 1.7655563354492188
Epoch: 0 / 100, Batch: 308 (0 / 39552), Elapsed time: 210.69 mins
Enc Loss = 190.08, KL Divergence = 156.75, Reconstruction Loss = 0.02, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 211.36 mins
Memory Use (GB): 1.7652511596679688
Epoch: 0 / 100, Batch: 309 (0 / 39